In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from pysal.lib import weights
from pysal.explore import esda
from pysal.model import spreg

# Cargar el archivo shapefile
gdfm = gpd.read_file("2_Espacial/CartografiasDatos_Python/cartografias/Munic04_ESP.shp")

# Crear matriz de pesos espaciales con la regla de la Reina
wq = weights.contiguity.Queen.from_dataframe(gdfm)
wq.transform = "R"

# MODELO (A): Regresión OLS
modelo_A = spreg.OLS(
    gdfm[["TASA_PARO"]].values,  # Variable dependiente
    gdfm[["RENTPCAP07"]].values,  # Variable independiente
    name_y="TASA_PARO",  # Nombre de la variable dependiente
    name_x=["RENTA_PERCAPITA"]  # Nombre de la variable independiente
)

# Guardar los residuos en el DataFrame
gdfm["residual"] = modelo_A.u

# Calcular el Índice de Moran sobre los residuos
moran = esda.moran.Moran(gdfm["residual"], wq)
print("I de Moran (Modelo A):", moran.I.round(3))
print("p-valor:", moran.p_sim)

# MODELO (B): Modelo espacial de errores heterocedásticos
modelo_B = spreg.GM_Error_Het(
    gdfm[["TASA_PARO"]].values,  # Variable dependiente
    gdfm[["RENTPCAP07"]].values,  # Variable independiente
    w=wq,  # Matriz de pesos espaciales
    name_y="TASA_PARO",  # Nombre de la variable dependiente
    name_x=["RENTA_PERCAPITA"]  # Nombre de la variable independiente
)

# Guardar los residuos filtrados en el DataFrame
gdfm["mLagresidual"] = modelo_B.e_filtered

# Calcular el Índice de Moran sobre los residuos filtrados
moran = esda.moran.Moran(gdfm["mLagresidual"], wq)
print("I de Moran (Modelo B):", moran.I.round(3))
print("p-valor:", moran.p_sim)

c:\Users\Usuario\AppData\Local\Programs\Python\Python312\Lib\site-packages\spaghetti\network.py:41: FutureWarning: The next major release of pysal/spaghetti (2.0.0) will drop support for all ``libpysal.cg`` geometries. This change is a first step in refactoring ``spaghetti`` that is expected to result in dramatically reduced runtimes for network instantiation and operations. Users currently requiring network and point pattern input as ``libpysal.cg`` geometries should prepare for this simply by converting to ``shapely`` geometries.
  warnings.warn(dep_msg, FutureWarning, stacklevel=1)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_16380\1379473058.py:12: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  wq = weights.contiguity.Queen.from_dataframe(gdfm)
c:\Users\Usuario\AppData\Local\Programs\Python\Python312\Lib\site-packages\libpysal\weights\contiguity.py:347: UserWarning: The we

('WARNING: ', 816, ' is an island (no neighbors)')
('WARNING: ', 2536, ' is an island (no neighbors)')
('WARNING: ', 5326, ' is an island (no neighbors)')
('WARNING: ', 7061, ' is an island (no neighbors)')
('WARNING: ', 7160, ' is an island (no neighbors)')
('WARNING: ', 7229, ' is an island (no neighbors)')
('WARNING: ', 8106, ' is an island (no neighbors)')
('WARNING: ', 8107, ' is an island (no neighbors)')
I de Moran (Modelo A): 0.581
p-valor: 0.001
I de Moran (Modelo B): -0.066
p-valor: 0.001
